# O PROBLEMA

Você foi contratado(a) para uma consultoria, e seu trabalho envolve analisar os dados de preço do petróleo brent, que pode ser encontrado no site do ipea. Essa base de dados histórica envolve duas colunas: data e preço (em dólares).

Um grande cliente do segmento pediu para que a consultoria desenvolvesse um dashboard interativo e que gere insights relevantes para tomada de decisão. Além disso, solicitaram que fosse desenvovido um modelo de Machine Learning para fazer o forecasting do preço do petróleo.

Seu objetivo é:

- Criar um dashboard interativo com ferramentyas de sua escolha.
- Seu dashboard deve fazer parte de um storytelling que traga insights relevantes sobre a variação do preço do petróleo, como situações geopolíticas, crises econômicas, demanda global por energia e etc. Isso pode te ajudar com seu modelo. É obrigatório que você traga pelo menos 4 insights neste desafio.
- Criar um modelo de Machine Learning que faça a previsão do preço do petróleo diariamente (lembre-se de time series). Esse modelo deve estar contemplado em seu storytelling e deve conter o código que você trabalhou, analisando as performances do modelo.
- Criar um plano para fazer o deploy em produção do modelo, com as ferramentas que são necessárias.
- Faça o MVP do seu modelo em produção usando o Streamlit.

Link para a base de dados: http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view

## Metodologia

<img src = 'img/CRISP-DM.png'>

Para o desenvolvimento deste projeto, optou por seguir a metodologia CRISP-DM (CRoss Industry Standard Process for Data Mining), largamente utilizada em projetos de dados. A CRISP-DM é composta por 6 etapas, são elas: 

- Análise do negócio (business understanding): essa etapa dedica-se ao entendimento do negócio, isto é, sua mercadoria, público alvo e quais são as estratégias do setor;

- Análise dos dados (data understanding): uma vez realizada a etapa anterior, inicia-se o entendimento dos dados, que compreende a seleção daqueles que são uteis à resolução da problemática requerida, bem como seu estado (fontes, formatos, etc);

- Preparação dos dados (data preparation): essa etapa dedica-se ao pré-processamento dos dados, em acordo com o que é solicitado pelas soluções que se pretende implementar;

- Modelagem (modeling): a modelagem é a etapa na qual se extrai informações de valor sobre os dados, possibilitando a geração de insights úteis ao negócio, capazes de solucionar a problemática;

- Avaliação (evaluation): momento de avaliação do desempenho do modelo aplicado, isto é, verifica-se se esse respondeu as questões levantadas satisfatoriamente;

- Implementação (deployment): por fim, a implementação é a etapa em que se disponibiliza os resultados e insights obtidos as partes interessadas, comumente por meio de uma ferramenta de data viz ou relatório.

## Análise do negócio

## Análise dos dados

Os dados utilizados neste projeto estão disponibilizados em uma tabela no site do IPEA (Instituto de Pesquisa Econômica Aplicada) e serão importados por meio da biblioteca pandas do python. A tabela disponibiliza os preços por barril do petróleo bruto tipo Brent, não incluindo despesa de frete e seguro. Os dados serão então exportados para um arquivo em formato csv e a partir do qual serão trabalhados, evitando problemas de indisponibilidade na fonte.

## Preparação dos dados

### Importando bibliotecas

In [168]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Importando dados

In [170]:
df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', skiprows=1, thousands='.', decimal=',')
df = df[0]
df.head()

,0,1
0,20/11/2023,83.25
1,17/11/2023,81.22
2,16/11/2023,77.73
3,15/11/2023,82.40
4,14/11/2023,84.20


In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       11049 non-null  object 
 1   1       11049 non-null  float64
dtypes: float64(1), object(1)
memory usage: 172.8+ KB


### Tratando dados

In [172]:
#verificando ocorrência de dados nulos
df.isna().sum()

0    0
1    0
dtype: int64

In [173]:
#verificando valores duplicados
df.duplicated().sum()

0

In [174]:
#verificando ocorrência de espaçamentos
espacamento_encontrado = False

for column in df.columns:
    for index, value in df[column].items():
        if isinstance(value, str) and value.isspace():
            print(f'Espaçamento encontrado na coluna "{column}", linha {index}.')
            espacamento_encontrado = True

if not espacamento_encontrado:
    print('Não há espaçamento nos dados.')

Não há espaçamento nos dados.


In [175]:
#renomeando colunas
df.rename(columns={0: 'data', 1: 'preco'}, inplace=True)
df.head()

,data,preco
0,20/11/2023,83.25
1,17/11/2023,81.22
2,16/11/2023,77.73
3,15/11/2023,82.40
4,14/11/2023,84.20


In [176]:
#alterando tipo da coluna de data
df['data'] = df['data'].str.replace('/', '-')
df['data'] = pd.to_datetime(df['data'], format='%d-%m-%Y')

In [177]:
df.head()

,data,preco
0,2023-11-20,83.25
1,2023-11-17,81.22
2,2023-11-16,77.73
3,2023-11-15,82.40
4,2023-11-14,84.20


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   data    11049 non-null  datetime64[ns]
 1   preco   11049 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 172.8 KB


**Exportando dados para arquivo csv**

In [179]:
df.to_csv('dados/dados.csv', index=False)

**Importando dados do arquivo csv**

In [180]:
df = pd.read_csv('dados/dados.csv')
df.head()

,data,preco
0,2023-11-20,83.25
1,2023-11-17,81.22
2,2023-11-16,77.73
3,2023-11-15,82.40
4,2023-11-14,84.20


In [181]:
#transformando coluna data em índice
df.set_index('data', inplace=True)
df.head()

,preco
data,
2023-11-20,83.25
2023-11-17,81.22
2023-11-16,77.73
2023-11-15,82.40
2023-11-14,84.20


### EDA

**Estatísticas descritivas**

**Distribuição das variáveis**

#### Decompondo a série

**Analisando sazonalidade**

**Visualizando médias móveis da série**

**Suavizando e diferenciando a série**

#### Teste estatístico de estacionariedade para a variável target

Ao analisar os dados de uma time series é preciso garantir que estes são estacionários, ou seja, que a time series possui um comportamento ao longo do tempo e que possui uma alta probabilidade de seguir este mesmo comportamento no futuro. Em econometria, o teste de estacionariedade, também conhecido como Augmented Dickey-Fuller (ADF) é uma técnica para testar a hipóteseda estacionariedade da time series.

>H0 - Hipótese Nula (não é estacionária)

>H1 - Hipótese Alternativa (rejeita a hipótese nula)

**p-valor <= 0.05 rejeitamos H0 com um nível de confiança de 95%**

#### ACF (autocorrelação) e PACF (autocorrelação parcial)

São duas técnicas estatísticas muito utilizadas na análise de séries temporais.

A ACF mede o grau de correlação entre os valores passados e presentes de uma série temporal, para diferentes defasagens ou lags. Já a PACF mede essa mesma correlação, porém removendo a influência das defasagens anteriores.

## Modelagem

## Avalição

## Implementação